In [1]:
import torch.utils.data as data
import pandas as pd
import os
import image_utils
import cv2
from torchvision import transforms
import torch
import random
import torchvision
from IPython.display import display
pil=torchvision.transforms.ToPILImage()#将tensor数据转换成图片

class MyDataSet(data.Dataset):
    def __init__(self,phase="train",transform=None,basic_aug=True):
        self.phase=phase
        self.transform=transform
        self.basic_aug=basic_aug
        
        NAME_COLUMN=0 #图片名字
        LABEL_COLUMN=1 #图片标签

        df=pd.read_csv("datasets/raf-basic/EmoLabel/list_patition_label.txt",sep=' ',header=None)
        if phase=="train":
            dataset=df[df[NAME_COLUMN].str.startswith('train')]
        else:
            dataset=df[df[NAME_COLUMN].str.startswith('test')]
        file_names=dataset.iloc[:,NAME_COLUMN].values    #所有行获得第一列的文件名字
        self.label=dataset.iloc[:,LABEL_COLUMN].values-1 #  0:Surprise, 1:Fear, 2:Disgust, 3:Happiness, 4:Sadness, 5:Anger, 6:Neutral

        self.file_paths=[]#使用对齐过的图片进行训练、测试

        for f in file_names:
            f=f.split(".")[0] #只要文件名字，不要后缀名
            f=f+"_aligned.jpg" #使用对齐后的图片
            path=os.path.join('datasets/raf-basic/Image/aligned',f)
            self.file_paths.append(path)

        self.aug_func=[image_utils.flip_image,image_utils.add_gaussian_noise]

    def __len__(self):
        return len(self.file_paths)   #返回一共有多少张图片

    def __getitem__(self,idx):   
        path=self.file_paths[idx]
        image=cv2.imread(path)
        # display(pil(image))
        image = image[:, :, ::-1]  #读取该张图片并将其转换为RGB格式，原格式为#BGR
        label=self.label[idx]  #得到该张图片对应的标签

        #如果是训练阶段，就对图片进行随机增强
        if self.phase=="train":
            if self.basic_aug and random.uniform(0,1)>0.5:
                #即如果basic_aug为真并且随机数大于0.5，就对该张图片进行随机增强
                index=random.randint(0,1) #随机取0或者取1
                image=self.aug_func[index](image) #取增强中的随机一种方法进行图片增强
        
            #然后再对图片进行预处理
        if self.transform is not None:
            #一系列变换，根据均值标准差进行归一化,随机翻转、随机遮挡
            image=self.transform(image)

        return image,label   #返回处理过后的图片数据、图片标签以及图片对应的位置




下面开始加载数据集

In [5]:
train_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(scale=(0.02,0.25))])
train_dataset=MyDataSet(phase='train',transform=train_transforms,basic_aug=True)
train_loader=torch.utils.data.DataLoader(train_dataset,
                                            batch_size=64,
                                            num_workers=4,
                                            shuffle=True,
                                            pin_memory=True)
test_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])        
test_dataset=MyDataSet(phase='test',transform=test_transforms,basic_aug=True)
test_loader=torch.utils.data.DataLoader(test_dataset,
                                        batch_size=64,
                                        num_workers=4,
                                        shuffle=False,
                                        pin_memory=True)

下面开始构建网络

In [4]:
from einops.layers.torch import Rearrange
from torch import nn
import torch
import einops
from torchvision import transforms

class VisionTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        #然后开始定义该网络涉及到的一些需要用的函数
        self.patch_embedding=nn.Sequential(
            Rearrange("b c (pw w) (ph h) -> b c (pw ph) (w h)",pw=16,ph=16),#将得到的每个小图片的长乘以宽的像素展平，得到256*196的向量
            nn.Linear(14*14,14*14), #每张小图片的向量为196，输出也为196
        )
        # shape: (batch,1, 7*7)
        #通过这样的方式就能将改变量加入到该模型的变量中，然后才能自动进行求导
        self.class_token=nn.parameter.Parameter(torch.randn(1,3,1,14*14))
        # shape: (batch, 4*4+1, 7*7)
        self.position_encodings=nn.parameter.Parameter(
            torch.randn((1,3,16*16+1,14*14))
        )
        #使用python自带的transfoermerEncoder结构
        #d_model是embedding向量的维数，设置它为7*7=49
        #nhead是MSA中self attention的个数，设置nhead=7
        #dim_feedforward是隐藏层的隐藏节点个数
        #btach_size是将张量的第一维指定为batch
        #3:the number of sub-encoder-layers in the encoder (required).
        self.transformer_encoder=nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=14*14,nhead=14,dim_feedforward=256,batch_first=True
            ),
            3,
        )
        #最后进行分类的MLP层，由于有十个类，因此需要将最后输出结果变为7个
        #nn.LayerNorm是对向量进行归一化，14*14表示对行进行归一化
        self.mlp_head=nn.Sequential(nn.LayerNorm(3*14*14),nn.Linear(3*14*14,7))

    #下面将定义该网络的前向传播过程
    def forward(self,x):
        x=self.patch_embedding(x)#首先将该批图片进行embedding，即剪裁和展平、线性变换
        batch_size, _ , _,_=x.shape#将X的第一维数赋值给batch_size，即一次输入了多少张图片
        #给每张图片(batch_size张图片)矩阵在最后都上class_token，得到class_token
        class_token=einops.repeat(
            self.class_token,
            "() channels words features -> repeat channels words features",
            repeat=batch_size
        )
        x = torch.cat((class_token,x),dim=-2)#即在第负二维（行）上进行拼接，然后就变成了257行
        x += self.position_encodings #然后再将x加入position_encoding，但是它可以通过训练得到
        x=self.transformer_encoder(x)#接下来将加了class_token和position_embeeding的最终向量输入encode
        x=x[:,:,0]#只取class_token那部分进行预测，class_token在第一行
        x=einops.rearrange(x,"b c h->(b c) h")
        x=self.mlp_head(x)#利用class_token取进行预测
        return x#返回结果




下面开始构建训练模块

In [5]:
def train(model,optimizer,scheduler,criterion,epochs,train_loader):
    model.train()
    for epoch in range(epochs):
        print(f"第{epoch}轮开始了")
        for batch_idx,(x,y) in enumerate(train_loader):
            print(f"第{batch_idx}次开始了")
            optimizer.zero_grad()
            output=model(x)
            loss=criterion(output,y)
            loss.backward()
            optimizer.step()
            print(f"第{batch_idx}次结束了")

            if batch_idx%10==0:
                print(f"epoch:{epoch},i:{batch_idx*len(x)}/{len(train_loader.dataset)},loss:{loss.item()}")
        scheduler.step()

下面开始构建训练参数并训练

In [6]:
vision_transformer_model=VisionTransformer()
epochs=70
batch_size=64
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(vision_transformer_model.parameters(),lr=0.001)
scheduler=torch.optim.lr_scheduler.StepLR(optimizer,step_size=1,gamma=0.9)



In [7]:
train(
    vision_transformer_model,
    optimizer,
    scheduler,
    criterion,
    epochs,
    train_loader,
)

第0轮开始了
